In [1]:
import pennylane as qml
from pennylane import numpy as np
import QEMZMSEPC as qem

In [2]:
n_qubits = 4
dev = qml.device('default.mixed', wires=n_qubits)

# In the simulation, we target different rotation angles of the RX gate.
# If the ZMSEPC theory is correct,
# the values after error mitigation through the QEM-ZMSEPC method should be the same as the ideal values.

rotation_angle = [0, 0.5, 0.9, 1.3, 1.7, 2.1, 2.5]
p = 0.8 # p: Depolarization rate.
for rotation_angle_of_rx in rotation_angle:
    nqubitspaulimatrices = qem.NqubitsPauliMatrices(n_qubits)
    pauli_set_n_qubits = nqubitspaulimatrices.get_pauli_matrices_of_n_qubits()
    nqubitschannel = qem.NqubitsChannel(n_qubits, pauli_set_n_qubits)
    kraus_matrices_of_a_pauli_channel = nqubitschannel.nqubitsrandompaulichannel()
    # The operations and paras required to simulate a Trotter step quantum circuit are as follows.
    operations = ['RX', 'RX', 'RX', 'RX', 'CNOT', 'CNOT', 'RZ', 'RZ', 'CNOT', 'CNOT', 'CNOT', 'RZ', 'CNOT']
    rotation_angle_of_rz = -0.2
    paras = [[0, rotation_angle_of_rx], [1, rotation_angle_of_rx], [2, rotation_angle_of_rx], [3, rotation_angle_of_rx], [0, 1], [2, 3], [1, rotation_angle_of_rz], [3, rotation_angle_of_rz], [0, 1], [2, 3], [1, 2], [2, rotation_angle_of_rz], [1, 2]]
    # apply QEM-ZMSEPC method
    qemzmsepc = qem.QEMZMSEPC(n_qubits)
    z_ideal = qemzmsepc.circuit_output(operations=operations, paras=paras, dev=dev)
    print(f"z_ideal is {z_ideal}")
    z_unmitigated = qemzmsepc.circuit_output(operations=operations, paras=paras, p=p, dev=dev,
                        kraus_matrices_of_a_pauli_channel=kraus_matrices_of_a_pauli_channel,
                        need_gate_noise=True, need_measurement_noise=True)
    print(f"z_unmitigated is {z_unmitigated}")
    z_mitigated, _ = qemzmsepc.qemzmsepc(operations=operations, paras=paras, p=p,
                        kraus_matrices_of_a_pauli_channel=kraus_matrices_of_a_pauli_channel, dev=dev)
    print(f"z_mitigated is {z_mitigated}")
    print("------------------")

z_ideal is 1.0
z_unmitigated is 0.12426427597433702
z_mitigated is 1.0000000000000067
------------------
z_ideal is 0.5931327983656769
z_unmitigated is 0.317034477132074
z_mitigated is 0.593132798365679
------------------
z_ideal is 0.1493041506116879
z_unmitigated is 0.09813488116690983
z_mitigated is 0.14930415061169283
------------------
z_ideal is 0.0051202072280735166
z_unmitigated is 0.002277696671655987
z_mitigated is 0.005120207228076839
------------------
z_ideal is 0.0002755900039979539
z_unmitigated is 0.00018788140751478666
z_mitigated is 0.00027559000399659357
------------------
z_ideal is 0.06495850756506487
z_unmitigated is 0.04035808512920488
z_mitigated is 0.06495850756505626
------------------
z_ideal is 0.4119471515970569
z_unmitigated is 0.12476283749054376
z_mitigated is 0.411947151597065
------------------
